In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service


In [2]:
chromedriver = "chromedriver"
options = webdriver.ChromeOptions()
options.add_argument("--user-data-dir=./User_Data")
driver = webdriver.Chrome(chromedriver, options=options)

In [3]:

def get_contacts(driver):
    # Refresh contacts by opening new chat list
    chat_button = driver.find_element(By.XPATH, '//div[@title="New chat"]')
    chat_button.click()
    time.sleep(1)
    
    # Scrape contact names
    contacts_all = []
    import pdb
    
    contacts_panel = driver.find_elements_by_css_selector("div.x1y332i5.x1n2onr6")
    last_height = driver.execute_script("return arguments[0].scrollHeight", contacts_panel)
    
    while True:
        # Get all contact names in the view
        contacts = []
        contact_elements = driver.find_elements_by_css_selector('div.x10l6tqk span.x1rg5ohu')
        for contact in contact_elements:
            contact_name = contact.text
            if contact_name:  # Avoid empty names
                contacts.append(contact_name)
                
        contacts = [contact.text for contact in contact_elements if contact.text]
        
        contacts_all.extend(contacts)
        # Scroll down
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", contacts_panel)
        time.sleep(2)  # Adjust delay as needed

        # Check if we are at the bottom
        new_height = driver.execute_script("return arguments[0].scrollHeight", contacts_panel)
        if new_height == last_height:  # No new contacts loaded
            break
        last_height = new_height
    
    # Close the contacts panel
    close_btn = driver.find_element(By.XPATH, '//span[@data-icon="back"]')
    close_btn.click()
    
    contact_elements = driver.find_elements_by_css_selector('div.x10l6tqk span.x1rg5ohu')

    
    # Close the contact list popup
    
    return contacts

def send_message_to_contacts(driver,message, image_path):
    # Setup the Chrome driver

    
    # Open WhatsApp Web
    driver.get("https://web.whatsapp.com")
    input("Scan QR code and press Enter after logging in...")

    # Get all contacts
    contacts = get_contacts(driver)
    print("Contacts found:", contacts)

    for contact in contacts:
        # Search and select contact
        try:
            search_box = driver.find_element_by_css_selector('.x1n2onr6.xh8yej3.lexical-rich-text-input')
            search_box.click()
            search_box.send_keys(contact)
            time.sleep(2)
            search_box.send_keys(Keys.ENTER)

            # Send the message
            message_box = driver.find_element(By.XPATH, '//div[@title="Type a message"]')
            message_box.send_keys(message + Keys.ENTER)
            time.sleep(2)

            # Send the image
            attach_btn = driver.find_element(By.XPATH, '//div[@title="Attach"]')
            attach_btn.click()
            time.sleep(1)

            image_input = driver.find_element(By.XPATH, '//input[@accept="image/*,video/mp4,video/3gpp,video/quicktime"]')
            image_input.send_keys(image_path)

            time.sleep(2)  # Wait for image to load
            send_btn = driver.find_element(By.XPATH, '//span[@data-icon="send"]')
            send_btn.click()
            time.sleep(3)

            # Delete the image using "Delete for me" option
            driver.find_element(By.XPATH, '//span[@data-icon="menu"]').click()
            time.sleep(1)
            delete_btn = driver.find_element(By.XPATH, '//div[@title="Delete message"]')
            delete_btn.click()
            time.sleep(1)
            only_me_btn = driver.find_element(By.XPATH, '//div[@title="Delete for me"]')
            only_me_btn.click()
        except StaleElementReferenceException:
            pass

    driver.quit()

In [ ]:
# Usage
message = "Hello! This is a test message."
image_path = "Ddiwali.jpeg"  # Replace with the path to your image
send_message_to_contacts(driver,message, image_path)